In [32]:
library(rvest)
library(purrr)
library(tidyverse)
library(glue)
library(magrittr)

# Web scraping #

## Part 1: scraping animal information

In [9]:
animals <- tibble()
links_html <- read_html("https://a-z-animals.com/animals/")
# scrape the html script from https://a-z-animals.com/animals/ as the main entrance
links_li <- links_html %>%
  html_nodes('[class^="az-phobia-link"]')
# there are 593 animals in this page
# use html_nodes to find all these links html tags with the css class name beginning with az-phobia-link
for(index in 1:length(links_li)){
# from link 1 to link 593
  animal_name <- links_li[index] %>%
    html_node("a") %>%
    html_attr("href")
  url <- glue("https://a-z-animals.com{animal_name}")
# combine the basic url and animal name as the new url link
  raw_response <- read_html(url)
  table_html <- raw_response %>%
    html_node(".az-facts")
# extract the table element with the css class named az-facts
  table_tr <- table_html %>%
    html_nodes("tr")
# find the tr tag in another loop of the table
  new_row <- tibble(index = numeric())
  new_row <- new_row %>% add_row(index = index)
# add an index (numeric ID) column to the data frame
  for(i in 1:length(table_tr)){
    attribute <- table_tr[i] %>%
      html_nodes("a") %>% html_text()
    table_td <- table_tr[i] %>%
      html_nodes("td")
    if(length(table_td) < 2)
      next
    value <- table_td[2] %>% html_text()
# two columns are controlled by td tag 
# the first column is the attribute and the second is the value of that attribute
    new_row <- new_row %>% add_column(!!attribute:=value)
# put the attribute as the column name and the value as the cell and add a new column to a data frame
  }
  if(new_row$index == 1){
    animals <- new_row
  }else{
    animals <- animals %>%
      full_join(new_row)
# add new columns to a data frame
  }
  Sys.sleep(1)
# pause the program for one second in every loop 
}
animals %>% write_csv("a-z-animals.csv")

Joining, by = c("index", "Common Name", "Group", "Colour", "Average Litter Size")
Joining, by = c("index", "Kingdom", "Phylum", "Class", "Order", "Family", "Genus", "Scientific Name", "Common Name", "Group", "Number Of Species", "Location", "Habitat", "Colour", "Skin Type", "Weight", "Top Speed", "Diet", "Prey", "Predators", "Lifestyle", "Group Behaviour", "Lifespan", "Age Of Sexual Maturity", "Name Of Young", "Conservation Status", "Estimated Population Size", "Biggest Threat", "Most Distinctive Feature", "Fun Fact")
Joining, by = c("index", "Common Name", "Group", "Colour", "Average Litter Size", "Origin", "Average Weight", "Average Lifespan", "Type", "Temperament", "Training", "Distinctive Features")
Joining, by = c("index", "Common Name", "Group", "Colour", "Average Litter Size", "Origin", "Average Weight", "Average Lifespan", "Type", "Temperament", "Training", "Distinctive Features", "Average Size (H)")
Joining, by = c("index", "Kingdom", "Phylum", "Class", "Order", "Family", "Gen

scrape each animal's name and join them with a-z-animals data frame together

In [4]:
url_titles <- "https://a-z-animals.com/animals/" # this is the base url from where the scraping starts
page_title <- read_html(url_titles)

In [7]:
title <-
page_title %>%
  html_nodes('[class^="az-phobia-link"]') %>%
  html_text()
animal_name_df <- tibble(Name = title) 

In [9]:
animal_name <- as.matrix(animal_name_df)
animal_name <- as.vector(animal_name)

In [10]:
animals_df <- read_csv(file = "a-z-animals.csv")

Parsed with column specification:
cols(
  .default = col_character(),
  index = col_double(),
  `Number Of Species` = col_number(),
  `Average Spawn Size` = col_number(),
  `Litter Size` = col_double()
)
See spec(...) for full column specifications.


In [11]:
a_z_annimals <- add_column(animals_df, Name = animal_name, .after = 1)

In [13]:
a_z_annimals %>% write_csv("A-Z animals.csv")

## Part 2: scraping animal images

In [2]:
animal_images <- tibble(animal_name=character(), image_url=character(), file_name=character())
animals <- tibble()
links_html <- read_html("https://a-z-animals.com/animals/")
links_li <- links_html %>%
  html_nodes('[class^="az-phobia-link"]')
for(index in 1:length(links_li)){
  animal_name <- links_li[index] %>%
    html_node("a") %>%
    html_attr("href")
  url <- glue("https://a-z-animals.com{animal_name}")
  content_html <- read_html(url)
  images <- content_html %>% html_node(".content") %>% html_nodes("img")
  image_root <- images[1] %>% html_attr('src')
  image_url <- glue("https://a-z-animals.com{image_root}")
  filename <- unlist(strsplit(image_url, "/"))
  animal_name <- content_html %>% 
    html_node("main header") %>% 
    html_node("h1") %>% 
    html_text()
  animal_images <- animal_images %>% add_row(animal_name=animal_name,image_url=image_url,file_name=filename[length(filename)])
  if(file.exists(paste0("animal-images/", destfile = filename[length(filename)]))){
    print(filename[length(filename)])
    next
  }
  download.file(url = image_url,paste0("animal-images/", destfile = filename[length(filename)]), mode="wb")
  
  Sys.sleep(1)
}
animal_images %>% write_csv("a-z-animals-image.csv")

[1] "aardvark_1.jpg"
[1] "abyssinian2.jpg"
[1] "adelie_penguin2.jpg"
[1] "affenpinscher_dog.jpeg"
[1] "afghan_hound1.jpg"
[1] "african_bush_elephant1.jpg"
[1] "african_civet4.jpg"
[1] "african_clawed_frog7.jpg"
[1] "african_elephant.jpg"
[1] "african_palm_civet.jpg"
[1] "african_penguin3.jpg"
[1] "african_tree_toad.jpg"
[1] "african_wild_dog4.jpg"
[1] "ainu_dog.jpg"
[1] "airedale_terrier.jpg"
[1] "akbash_dog1.jpg"
[1] "akita_dog.jpg"
[1] "alaskan_malamute6.jpg"
[1] "albatross3.jpg"
[1] "aldabra_giant_tortoise5.jpg"
[1] "alligator6.jpg"
[1] "alpine_dachsbracke1.jpg"
[1] "american_bulldog.jpg"
[1] "american_cocker_spaniel1.jpg"
[1] "american_coonhound2.jpg"
[1] "american_eskimo_dog3.jpg"
[1] "american_foxhound2.jpg"
[1] "american_pit_bull_terrier.jpg"
[1] "american_staffordshire_terrier2.jpg"
[1] "american_water_spaniel1.jpg"
[1] "anatolian_shepherd_dog.jpg"
[1] "angelfish11.jpg"
[1] "ant.jpg"
[1] "anteater3.jpg"
[1] "antelope3.jpg"
[1] "appenzeller_dog1.jpg"
[1] "arctic_fox10.jpg"
[1] "

# Data wrangling #

In [33]:
az_animals <- read_csv(file = "A-Z animals.csv")

Parsed with column specification:
cols(
  .default = col_character(),
  index = col_double(),
  `Number Of Species` = col_double(),
  `Average Spawn Size` = col_double(),
  `Litter Size` = col_double()
)
See spec(...) for full column specifications.


Normalize the colunm name

In [34]:
az_animals<- az_animals %>% rename(Index = index)

In [35]:
az_animals_image <- read_csv(file = "a-z-animals-image.csv")

Parsed with column specification:
cols(
  animal_name = col_character(),
  image_url = col_character(),
  file_name = col_character()
)


Normalize the colunm name

In [36]:
az_animals_image <- az_animals_image %>% rename(Name = animal_name,
                           "Image URL" = "image_url",
                           "File Name" = "file_name")

Join animal information and animal images together!

In [37]:
join_animals <- az_animals %>%
  full_join(az_animals_image)

Joining, by = "Name"


In [38]:
join_animals

Index,Name,Kingdom,Phylum,Class,Order,Family,Genus,Scientific Name,Common Name,⋯,Average Size,Optimum pH Level,Favourite Food,Special Features,Size,Extinct,Litter Size,Subfamily,Image URL,File Name
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
1,Aardvark,Animalia,Chordata,Mammalia,Tubulidentata,Orycteropodidae,Orycteropus,Orycteropus afer,Aardvark,⋯,NA,NA,NA,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/aardvark_1.jpg,aardvark_1.jpg
2,Abyssinian,NA,NA,NA,NA,NA,NA,NA,Abyssinian,⋯,NA,NA,NA,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/abyssinian2.jpg,abyssinian2.jpg
3,Adelie Penguin,Animalia,Chordata,Aves,Sphenisciformes,Spheniscidae,Pygoscelis,Pygoscelis adeliae,Adelie Penguin,⋯,NA,NA,NA,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/adelie_penguin2.jpg,adelie_penguin2.jpg
4,Affenpinscher,NA,NA,NA,NA,NA,NA,NA,Affenpinscher,⋯,NA,NA,NA,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/affenpinscher_dog.jpeg,affenpinscher_dog.jpeg
5,Afghan Hound,NA,NA,NA,NA,NA,NA,NA,Afghan Hound,⋯,NA,NA,NA,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/afghan_hound1.jpg,afghan_hound1.jpg
6,African Bush Elephant,Animalia,Chordata,Mammalia,Proboscidea,Elephantidae,Loxodonta,Loxodonta africana africana,African Bush Elephant,⋯,NA,NA,NA,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/african_bush_elephant1.jpg,african_bush_elephant1.jpg
7,African Civet,Animalia,Chordata,Mammalia,Carnivora,Viverridae,Civettictis,Civettictis civetta,African Civet,⋯,NA,NA,NA,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/african_civet4.jpg,african_civet4.jpg
8,African Clawed Frog,Animalia,Chordata,Amphibia,Anura,Pipidae,Xenopus,Xenopus laevis,African Clawed Frog,⋯,NA,NA,NA,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/african_clawed_frog7.jpg,african_clawed_frog7.jpg
9,African Forest Elephant,Animalia,Chordata,Mammalia,Proboscidea,Elephantidae,Loxodonta,Loxodonta cyclotis,African Forest Elephant,⋯,NA,NA,NA,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/african_elephant.jpg,african_elephant.jpg


check about the types of the columns and the missing values

In [39]:
join_animals %>% glimpse()

Observations: 593
Variables: 64
$ Index                       <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13…
$ Name                        <chr> "Aardvark", "Abyssinian", "Adelie Penguin…
$ Kingdom                     <chr> "Animalia", NA, "Animalia", NA, NA, "Anim…
$ Phylum                      <chr> "Chordata", NA, "Chordata", NA, NA, "Chor…
$ Class                       <chr> "Mammalia", NA, "Aves", NA, NA, "Mammalia…
$ Order                       <chr> "Tubulidentata", NA, "Sphenisciformes", N…
$ Family                      <chr> "Orycteropodidae", NA, "Spheniscidae", NA…
$ Genus                       <chr> "Orycteropus", NA, "Pygoscelis", NA, NA, …
$ `Scientific Name`           <chr> "Orycteropus afer", NA, "Pygoscelis adeli…
$ `Common Name`               <chr> "Aardvark", "Abyssinian", "Adelie Penguin…
$ `Other Name(s)`             <chr> "Antbear, Earth Pig", NA, NA, NA, NA, "Af…
$ Group                       <chr> "Mammal", "Cat", "Bird", "Dog", "Dog", "M…
$ `Number Of Species

Select more useful information

In [40]:
animal_1 <- join_animals %>%
  select(Index, Name, Group, Location, Habitat, 
         `Conservation Status`, `Estimated Population Size`, `Biggest Threat`,
        `Image URL`, `File Name`)
animal_1

Index,Name,Group,Location,Habitat,Conservation Status,Estimated Population Size,Biggest Threat,Image URL,File Name
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Aardvark,Mammal,Sub-Saharan Africa,Sandy and clay soil,Least Concern,Unknown,Habitat loss,https://a-z-animals.com/media/animals/images/470x370/aardvark_1.jpg,aardvark_1.jpg
2,Abyssinian,Cat,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/abyssinian2.jpg,abyssinian2.jpg
3,Adelie Penguin,Bird,Coastal Antarctica,Antarctic land and ocean,Least Concern,5 million,Rapid ice melt,https://a-z-animals.com/media/animals/images/470x370/adelie_penguin2.jpg,adelie_penguin2.jpg
4,Affenpinscher,Dog,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/affenpinscher_dog.jpeg,affenpinscher_dog.jpeg
5,Afghan Hound,Dog,NA,NA,NA,NA,NA,https://a-z-animals.com/media/animals/images/470x370/afghan_hound1.jpg,afghan_hound1.jpg
6,African Bush Elephant,Mammal,central and southern Africa,"Forest, savannah and flood plains",Threatened,"300,000",Poaching and habitat loss,https://a-z-animals.com/media/animals/images/470x370/african_bush_elephant1.jpg,african_bush_elephant1.jpg
7,African Civet,Mammal,across sub-Saharan Africa,Tropical rainforest,Least Concern,Abundant,Habitat loss,https://a-z-animals.com/media/animals/images/470x370/african_civet4.jpg,african_civet4.jpg
8,African Clawed Frog,Amphibian,eastern and southern Africa,Warm stagnant water with grassland,Least Concern,Abundant,Water pollution,https://a-z-animals.com/media/animals/images/470x370/african_clawed_frog7.jpg,african_clawed_frog7.jpg
9,African Forest Elephant,Mammal,central and southern Africa,"Forest, savannah and flood plains",Endangered,"200,000",Poaching and habitat loss,https://a-z-animals.com/media/animals/images/470x370/african_elephant.jpg,african_elephant.jpg


In [42]:
write_csv(animal_1, "A-Z animals & images.csv")

use the variable `Conservation Status` to arrange dataframe

In [20]:
animal_2 <- arrange(animal_1, `Conservation Status`)
animal_2

Index,Name,Group,Location,Habitat,Conservation Status,Estimated Population Size,Biggest Threat,Image URL,File Name
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
122,Camel,Mammal,Throughout the Middle East,Arid desert and scrubland,Common,20 million,Drought,https://a-z-animals.com/media/animals/images/470x370/camel2.jpg,camel2.jpg
84,Black Rhinoceros,NA,NA,"Tropical bushland, grassland and savannas",Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/black_rhinoceros1.jpg,black_rhinoceros1.jpg
99,Bornean Orang-utan,Mammal,Borneo,Lowland forest and peat-swamps,Critically Endangered,"13,500",Habitat loss,https://a-z-animals.com/media/animals/images/470x370/bornean_orang-utan1.jpg,bornean_orang-utan1.jpg
100,Borneo Elephant,NA,NA,Rainforest and tropical woodland,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/borneo_elephant.jpg,borneo_elephant.jpg
160,Cottontop Tamarin,NA,NA,Lowland tropical forest,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/cottontop_tamarin3.jpg,cottontop_tamarin3.jpg
169,Cross River Gorilla,NA,NA,Rainforest and dense jungle,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/cross_river_gorilla.jpg,cross_river_gorilla.jpg
199,Eastern Gorilla,NA,NA,Tropical forest and jungles in mountainous regions,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/eastern_gorilla.jpg,eastern_gorilla.jpg
249,Gharial,Reptile,Northern India and Nepal,Wide and calm slow-moving rivers,Critically Endangered,182,Habitat loss and hunting,https://a-z-animals.com/media/animals/images/470x370/gharial4.jpg,gharial4.jpg
322,Javan Rhinoceros,NA,NA,"Tropical bushland, grassland and rainforest",Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/javan_rhinoceros.jpg,javan_rhinoceros.jpg


use the variable `Conservation Status` to group dataframe

In [21]:
group_animal_2 <- animal_2 %>%
  group_by(`Conservation Status`) %>%
  tally()
group_animal_2

Conservation Status,n
<chr>,<int>
Common,1
Critically Endangered,19
Data Deficient,2
Endangered,73
Extinct,5
Least Concern,199
"Least Concern, Vulnerable, Critically Endangered",1
Near Threatened,7
Not Listed,4


filter out the missing data and the variable values that are not related to endangerment

In [22]:
animal_3 <- animal_2 %>%
  filter(`Conservation Status` != 'NA') %>%
  filter(`Conservation Status` != 'Common') %>%
  filter(`Conservation Status` != 'Data Deficient') %>%
  filter(`Conservation Status` != 'Not Listed') %>%
  filter(`Conservation Status` != 'Least Concern, Vulnerable, Critically Endangered')
animal_3

Index,Name,Group,Location,Habitat,Conservation Status,Estimated Population Size,Biggest Threat,Image URL,File Name
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
84,Black Rhinoceros,NA,NA,"Tropical bushland, grassland and savannas",Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/black_rhinoceros1.jpg,black_rhinoceros1.jpg
99,Bornean Orang-utan,Mammal,Borneo,Lowland forest and peat-swamps,Critically Endangered,"13,500",Habitat loss,https://a-z-animals.com/media/animals/images/470x370/bornean_orang-utan1.jpg,bornean_orang-utan1.jpg
100,Borneo Elephant,NA,NA,Rainforest and tropical woodland,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/borneo_elephant.jpg,borneo_elephant.jpg
160,Cottontop Tamarin,NA,NA,Lowland tropical forest,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/cottontop_tamarin3.jpg,cottontop_tamarin3.jpg
169,Cross River Gorilla,NA,NA,Rainforest and dense jungle,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/cross_river_gorilla.jpg,cross_river_gorilla.jpg
199,Eastern Gorilla,NA,NA,Tropical forest and jungles in mountainous regions,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/eastern_gorilla.jpg,eastern_gorilla.jpg
249,Gharial,Reptile,Northern India and Nepal,Wide and calm slow-moving rivers,Critically Endangered,182,Habitat loss and hunting,https://a-z-animals.com/media/animals/images/470x370/gharial4.jpg,gharial4.jpg
322,Javan Rhinoceros,NA,NA,"Tropical bushland, grassland and rainforest",Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/javan_rhinoceros.jpg,javan_rhinoceros.jpg
381,Monte Iberia Eleuth,NA,NA,Water around Mount Iberia,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/monte_iberia_eleuth3.jpg,monte_iberia_eleuth3.jpg


use the variable `Conservation Status` to group dataframe

In [23]:
group_animal_3 <- animal_3 %>%
  group_by(`Conservation Status`) %>%
  tally()
group_animal_3

Conservation Status,n
<chr>,<int>
Critically Endangered,19
Endangered,73
Extinct,5
Least Concern,199
Near Threatened,7
Threatened,116
Vulnerable,25


continue to filter out the variable values that are not related to endangerment

In [24]:
animal_4 <- animal_3 %>%
  filter(`Conservation Status` != 'Least Concern') %>%
  filter(`Conservation Status` != 'Near Threatened') %>%
  filter(`Conservation Status` != 'Threatened')
animal_4

Index,Name,Group,Location,Habitat,Conservation Status,Estimated Population Size,Biggest Threat,Image URL,File Name
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
84,Black Rhinoceros,NA,NA,"Tropical bushland, grassland and savannas",Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/black_rhinoceros1.jpg,black_rhinoceros1.jpg
99,Bornean Orang-utan,Mammal,Borneo,Lowland forest and peat-swamps,Critically Endangered,"13,500",Habitat loss,https://a-z-animals.com/media/animals/images/470x370/bornean_orang-utan1.jpg,bornean_orang-utan1.jpg
100,Borneo Elephant,NA,NA,Rainforest and tropical woodland,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/borneo_elephant.jpg,borneo_elephant.jpg
160,Cottontop Tamarin,NA,NA,Lowland tropical forest,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/cottontop_tamarin3.jpg,cottontop_tamarin3.jpg
169,Cross River Gorilla,NA,NA,Rainforest and dense jungle,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/cross_river_gorilla.jpg,cross_river_gorilla.jpg
199,Eastern Gorilla,NA,NA,Tropical forest and jungles in mountainous regions,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/eastern_gorilla.jpg,eastern_gorilla.jpg
249,Gharial,Reptile,Northern India and Nepal,Wide and calm slow-moving rivers,Critically Endangered,182,Habitat loss and hunting,https://a-z-animals.com/media/animals/images/470x370/gharial4.jpg,gharial4.jpg
322,Javan Rhinoceros,NA,NA,"Tropical bushland, grassland and rainforest",Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/javan_rhinoceros.jpg,javan_rhinoceros.jpg
381,Monte Iberia Eleuth,NA,NA,Water around Mount Iberia,Critically Endangered,NA,NA,https://a-z-animals.com/media/animals/images/470x370/monte_iberia_eleuth3.jpg,monte_iberia_eleuth3.jpg


In [29]:
group_animal_4 <- animal_4 %>%
  group_by(`Conservation Status`) %>%
  tally()
group_animal_4
summary(animal_4) 

Conservation Status,n
<chr>,<int>
Critically Endangered,19
Endangered,73
Extinct,5
Vulnerable,25


     Index           Name              Group             Location        
 Min.   :  9.0   Length:122         Length:122         Length:122        
 1st Qu.:167.5   Class :character   Class :character   Class :character  
 Median :340.5   Mode  :character   Mode  :character   Mode  :character  
 Mean   :323.2                                                           
 3rd Qu.:483.5                                                           
 Max.   :589.0                                                           
   Habitat          Conservation Status Estimated Population Size
 Length:122         Length:122          Length:122               
 Class :character   Class :character    Class :character         
 Mode  :character   Mode  :character    Mode  :character         
                                                                 
                                                                 
                                                                 
 Biggest Threat     